In [8]:
from collections import Counter
# from responses import responses, blank_spot
from user_functions import preprocess, compare_overlap, \
pos_tag, extract_nouns, compute_similarity
from QA import queries,responses,blank_spot

import spacy
word2vec = spacy.load('en_core_web_md')#Model selected here:https://spacy.io/models

exit_commands = ("quit", "goodbye", "exit", "no",'no,thanks!')

class ChatBot:
  
  #define .make_exit() below:
    def make_exit(self,user_message):
        for object in exit_commands:
            if object == user_message:
                print("Good bye!")
                return True

#define start_chat() below:
    def chat(self):
        user_message=input("Hi, what camping preferences do you have in mind?")
        if not self.make_exit(user_message):
            self.respond(user_message)
        else:
            return
        while not self.make_exit(user_message):
            user_message=input("Any other question?")
            if self.make_exit(user_message):
                return
            else:
                self.respond(user_message)

  #define .find_intent_match() below:
    def find_intent_match(self,responses,user_message):
        BoW_user_message=Counter(preprocess(user_message))
    #Counter creates a BoW model
    #Preprocess will tokenize the message
    #Can combine BoW and preprocess in one line
        BoW_responses=[Counter(preprocess(response)) for response in responses]
        similar_word_count_list=[]
        for response in BoW_responses:
            similar_word_count=compare_overlap(BoW_user_message,response)
            similar_word_count_list.append(similar_word_count)
            response_index= similar_word_count_list.index(max(similar_word_count_list))
        return responses[response_index]

  #define .respond() below:
    def respond(self,user_message):
        best_response=self.find_intent_match(responses,user_message)
        entity=self.find_entities(user_message)
#         input_message=input("Any other question?")#1/24 Commented out coz I think it's more reasonable to respond to user's response to the "speciality Q" first
        print(best_response.format(entity))#Assigning the slot in best_response to entity#e.g. best response: "{} includes habanero, so it is a bit spicy!" becomes  "ENTITY includes habanero, so it is a bit spicy!"

            
        #對方還沒回答時就給了根據上一個 user_message 配對而來的答案

    def find_intent_match(self,responses,user_message):
        BoW_user_message=Counter(preprocess(user_message))
        BoW_responses=[Counter(preprocess(response)) for response in responses]
        similar_word_count_list=[]
        for response in BoW_responses:
            similar_word_count=compare_overlap(BoW_user_message,response)
            similar_word_count_list.append(similar_word_count)
            response_index= similar_word_count_list.index(max(similar_word_count_list))
        return responses[response_index]
    
    #define .find_entities() below:
    def find_entities(self,user_message):
        message_nouns=extract_nouns(pos_tag(preprocess(user_message)))
        tokens=word2vec(" ".join(message_nouns))
        category=word2vec(blank_spot)
        word2vec_result=compute_similarity(tokens,category)#Select the word to fill the blank slot 
        word2vec_result.sort(key=lambda x:x[2])#Why is it possible that word2vec result is empty
        if word2vec_result:#CA: if len(word2vec_result) <1:
            return word2vec_result[-1][0]
        else:
            return blank_spot
        

Gbot=ChatBot()
Gbot.chat()

Hi, what camping preferences do you have in mind?desert sunset resort
joshua tree national park in california is a fantastic option for desert resort and stargazing due to its clear night skies.
Any other question?Rafting paradise
sure, you can try jellystone park paradise, which are known for family-friendly amenities like pools, playgrounds, and organized activities.
Any other question?offers snorkeling 
consider offers at cape hatteras national seashore in north carolina, where you can enjoy kayaking, snorkeling, and other water activities along the atlantic coast.
Any other question?no,thanks!
Good bye!
